# ESML - `AutoMLFactory` and `ComputeFactory`

## PROJECT + DATA CONCEPTS + ENTERPRISE Datalake Design + DEV->PROD MLOps
- `1)ESML Project`: The ONLY thing you need to remember is your `Project number` (and `BRONZE, SILVER, GOLD` concept )
   -  ...`read earlier notebook
## ENTERPRISE Deployment of Models & Governance - MLOps  at scale
- `3) DEV->TEST-PROD` (configs, compute, performance)
    - ESML has config for 3 environemnts: Easy DEPLOY model across subscriptions and Azure ML Studio workspaces 
        - Save costs & time: 
            - `DEV` has cheaper compute performance for TRAIN and INFERENCE (batch, AKS)
            - `DEV` has Quick-debug ML training (fast training...VS good scoring in TEST and PROD)
        - How? ESML `AutoMLFactory` and `ComputeFactory`
        - Where to config these?
            - settings/dev_test_prod/`dev_test_prod_settings.json`
            - settings/dev_test_prod/`train/*/automl/*`

# Azure ML Studio Workspace
- ESML will `Automap` and `Autoregister` Azure ML Datasets as: `IN, SILVER, BRONZE, GOLD`

In [1]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLDataset, ESMLProject

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../../../settings', you should copy template settings from '../settings'
p.active_model = 11
p.ws = p.get_workspace_from_config() #2) Load DEV or TEST or PROD Azure ML Studio workspace
p.inference_mode = False

In [ ]:
datastore = p.init()

# ESML `GOLD` Dataset

In [3]:
ds_01 = p.DatasetByName("ds01_diabetes")
print(ds_01.InData.name)
print(ds_01.Bronze.name)
print(ds_01.Silver.name)
#print(p.Gold.name)

M03_ds01_diabetes_train_IN
M03_ds01_diabetes_train_BRONZE
M03_ds01_diabetes_train_SILVER


In [4]:
df_01 = ds_01.Silver.to_pandas_dataframe() 

ds_02 = ds_01 = p.DatasetByName("ds02_other")
df_02 = ds_02.Silver.to_pandas_dataframe()
df_gold1_join = df_01.join(df_02) # left join -> NULL on df_02
print("Diabetes shape: ", df_01.shape)
print(df_gold1_join.shape)

Diabetes shape:  (185, 11)
(185, 19)


In [5]:
ds_gold_v1 = p.save_gold(df_01)

# Look at `GOLD` vLatest

In [6]:
import pandas as pd 
df = p.Gold.to_pandas_dataframe()
df.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,0.038076,0.05068,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,0.085299,0.05068,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
2,0.063504,0.05068,-0.001895,0.066630,0.090620,0.108914,0.022869,0.017703,-0.035817,0.003064,63.0
3,0.041708,0.05068,0.061696,-0.040099,-0.013953,0.006202,-0.028674,-0.002592,-0.014956,0.011349,110.0
4,0.027178,0.05068,0.017506,-0.033214,-0.007073,0.045972,-0.065491,0.071210,-0.096433,-0.059067,69.0


In [8]:
train, validate, test = p.split_gold_3(0.6, "Y") # Also registers the datasets in AZURE as M03_GOLD_TRAIN | M03_GOLD_VALIDATE | M03_GOLD_TEST

## 3) ESML TRAIN model -> See other notebook `esml_howto_2_train.ipynb`
- `AutoMLFactory, ComputeFactory`
- Get `Train COMPUTE` for `X` environment
- Get `Train Hyperparameters` for `X` environment (less crossvalidations in DEV etc)

## 4a) ESML Scoring compare: Promote model or not? Register
- `IF` newly trained model in `current` environment scores BETTER than existing model in `target` environment, then `new model` can be registered and promoted.
-  `ValidationSet` comparison of offline/previous `AutoML run` for `DEV` environment
- For `DEV`, `TEST` or `PROD` environment
- Future roadmap: Also include `TestSet SCORING` comparison

In [4]:
from baselayer_azure_ml_model import ESMLModelCompare

current_env = "dev" #p.dev_test_prod # dev
target_env = "test" # Does newly trained Model v3 in DEV, score better than Model v2 in TEST?
print("promote model in DEV to TEST? (move to other Azure ML Studio Workspace)")

compare = ESMLModelCompare(p)
promote,source_model_name,new_run_id,target_model_name, target_best_run_id,target_workspace,source_model = compare.compare_scoring_current_vs_new_model(target_env) # Compare DEV to TEST (or TEST to PROD)  (1min, 17sek VS 33sec)

print("SCORING DRIFT: If new model scores better in DEV (new data, or new code), we can promote this to TEST & PROD \n")
print("New Model: {} in environment {}".format(target_model_name, p.dev_test_prod))
print("Existing Model: {} in environment {}".format(source_model_name,target_env))

if (promote): # Can register="promote" a model in same workspace (test->test), or also register in OTHER Azure ML workspace (test->prod)
    if(p.dev_test_prod == target_env):
        compare.register_active_model(target_env,source_model) # if SAME workspace this brings more "metadata" faster to the model registration (will  register with EXPERIMENT and RUNID)
    else:
        compare.register_model_in_correct_ws(target_env) # if REMOTE target workspace we can get same metadata, BUT, just takes performancewise longer. More lookups to "source Run"

SCORING DRIFT: If new model scores better in DEV (new data, or new code), we can promote this to TEST & PROD 

Loading AutoML config settings from: dev
Compare model version in DEV with latest registered in TEST subscription/workspace

Package:azureml-automl-runtime, training version:1.30.0, current version:1.26.0
Package:azureml-core, training version:1.30.0, current version:1.26.0
Package:azureml-dataprep, training version:2.15.1, current version:2.13.2
Package:azureml-dataprep-native, training version:33.0.0, current version:32.0.0
Package:azureml-dataprep-rslex, training version:1.13.0, current version:1.11.2
Package:azureml-dataset-runtime, training version:1.30.0, current version:1.26.0
Package:azureml-defaults, training version:1.30.0, current version:1.26.0
Package:azureml-interpret, training version:1.30.0, current version:1.26.0
Package:azureml-pipeline-core, training version:1.30.0, current version:1.26.0
Package:azureml-telemetry, training version:1.30.0, current version:1.

# START 2) TEST env - `register a model` starting "offline", not an active training run?

### Alt 1) No ESMLProject dependency

In [10]:
import repackage
repackage.add("../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from baselayer_azure_ml import AutoMLFactory

ws = p.get_workspace_from_config() # Simulate you init a Azure ML Workspace
AutoMLFactory().register_active_model_in_ws(ws,"dev") # Simulate you register a model in Workspace

model.version 4
Model name AutoMLa4b60322a0 is registered.


Model(workspace=Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG'), name=AutoMLa4b60322a0, id=AutoMLa4b60322a0:4, version=4, tags={'run_id': 'AutoML_a4b60322-a808-4aa6-b5c8-4c5da22a4802', 'model_name': 'AutoMLa4b60322a0', 'trained_in_environment': 'dev', 'trained_in_workspace': 'msft-weu-DEV-eap-proj02_ai-amls'}, properties={})

### Alt 2) ESMLProject dependency: `ENVIRONMENT Self aware` and `config aware`
 - More `Future proof`: Features such as "able to register trained model in TARGET - from TEST to PROD without retraining"

In [11]:
sys.path.append(os.path.abspath("../common/"))  # NOQA: E402
from esml import ESMLDataset, ESMLProject
from baselayer_azure_ml import AutoMLFactory
from azureml.core import Workspace

ws = p.get_workspace_from_config()

p = ESMLProject() # Makes it "environment aware (dev,test,prod)", and "configuration aware"
p.init(ws) 
p.dev_test_prod = "dev"
# ....train model....

model = AutoMLFactory(p).register_active_model(p.dev_test_prod)

...
Using GEN2 as Datastore
ds01_diabetes
ds02_other

####### Automap & Autoregister - SUCCESS!
1) Auto mapped 2 ESML Dataset with registered Azure ML Datasets (potentially all 3: IN,BRONZE, SILVER) in Datastore project002lake 

Dataset 'ds01_diabetes' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files
Dataset 'ds02_other' status:
 - IN_Folder_has_files
 - BRONZE_Folder_has_files
 - SILVER_Folder_has_files

2) Registered each Dataset with suffixes (_IN_CSV, _BRONZE, _SILVER) 
 Tip: Use ESMLProject.Datasets list or .DatasetByName(myDatasetName) to read/write
#######
model.version 5
Model name AutoMLa4b60322a0 is registered.


Model(workspace=Workspace.create(name='msft-weu-DEV-eap-proj02_ai-amls', subscription_id='ca0a8c40-b06a-4e4e-8434-63c03a1dee34', resource_group='MSFT-WEU-EAP_PROJECT02_AI-DEV-RG'), name=AutoMLa4b60322a0, id=AutoMLa4b60322a0:5, version=5, tags={'run_id': 'AutoML_a4b60322-a808-4aa6-b5c8-4c5da22a4802', 'model_name': 'AutoMLa4b60322a0', 'trained_in_environment': 'dev', 'trained_in_workspace': 'msft-weu-DEV-eap-proj02_ai-amls'}, properties={})

### ..Model compared, promoted, register - ready for deployment

## 4b) ESML Loadtesting performance
- Using `GOLD_TEST` TestSet for AutoML to see which algorithm that is fastest, smallest size footprint
- For `DEV`, `TEST` or `PROD` environment

In [12]:
label = p.active_model["label"]
train, validate, test = p.split_gold_3() # Save as M03_GOLD_TRAIN | M03_GOLD_VALIDATE | M03_GOLD_TEST  # Alt: train_data, test_data = p.Gold.random_split(percentage=0.8, seed=223) 
test.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
207,0.01,-0.04,0.05,0.03,0.01,-0.01,0.03,-0.04,0.05,0.04,202.00
212,0.07,-0.04,0.00,0.04,0.05,0.03,0.07,-0.04,-0.00,0.02,73.00
295,-0.05,0.05,0.04,-0.04,-0.01,-0.01,0.01,-0.04,0.02,0.00,85.00
403,-0.02,-0.04,0.10,-0.01,-0.01,-0.02,-0.02,-0.00,0.06,0.04,275.00
251,-0.05,0.05,0.10,0.09,0.06,0.05,-0.06,0.11,0.08,0.04,243.00


## 5a) ESML Deploy ONLINE, to AKS -> See other notebook
- Deploy "offline" from old `AutoML run` for `DEV` environment
- To →  `DEV`, `TEST` or `PROD` environment

GOTO Notebook [`esml_howto_3_compare_and_deploy`](./esml_howto_3_compare_and_deploy)

## 5b) ESML `Deploy BATCH` pipeline
- Deploy same model "offline / previous" `AutoML Run` for `DEV` environment
- To →  `DEV`, `TEST` or `PROD` environment
